In [1]:
import json
import math
import gurobipy as gp
from gurobipy import Model, GRB, quicksum



In [ ]:
problem = 'tests/drone_problem_23.json'
with open(problem, 'r') as file:
    data = json.load(file)

# Extract data from JSON
drone_weight = data['drone_weight']
max_capacity = data['max_capacity']
battery_capacity = data['battery_capacity']
points = data['pontos']

points


In [3]:
n = len(points) - 2  
base_start = 0
base_end = n + 1


d = {}
for i in range(len(points)):
    for j in range(len(points)):
        if i != j:
            d[i, j] = math.sqrt((points[i]['x'] - points[j]['x'])**2 + (points[i]['y'] - points[j]['y'])**2)
        else:
            d[i, j] = 0


p = {}
for i in range(1, len(points) - 1):
    p[i] = points[i]['peso']


p[base_start] = 0
p[base_end] = 0


In [4]:
env = gp.Env(empty=True)
env.setParam('LICENSEID', 2548345)
env.start()
model=Model('drone')
model.setParam('TimeLimit', 500)
alpha = 1
beta = 1
x = model.addVars(len(points), len(points), vtype=GRB.BINARY, name="x")
omega = model.addVars(len(points), vtype=GRB.CONTINUOUS, name="omega")
e = model.addVars(len(points), len(points), vtype=GRB.CONTINUOUS, name="e")


#1. cada casa so vai para 1 lugar
for i in range(1, n + 1):
    model.addConstr(quicksum(x[i, j] for j in range(1, n + 2)) == 1, name=f"c1_{i}")
#2.nao vai da base para a base
model.addConstr(x[base_start, base_end] == 0, name="c2")
#3. apos voltar, nao sai
for j in range(len(points)):
    model.addConstr(x[base_end, j] == 0, name=f"c3_{j}")
#4. nao vai da casa para ela mesma
for i in range(len(points)):
    model.addConstr(x[i, i] == 0, name=f"c4_{i}")
#5. nao vai para base inicial
for i in range(len(points)):
    model.addConstr(x[i, base_start] == 0, name=f"c5_{i}")
#6. entra na base o mesmo numero de vezes que sai
model.addConstr(quicksum(x[i, base_end] for i in range(len(points))) == quicksum(x[base_start, j] for j in range(len(points))), name="c6")
#7. cada casa eh visitada 1 vez
for j in range(1, n + 1):
    model.addConstr(quicksum(x[i,j] for i in range(len(points))) == 1, name=f"c7_{j}")
#8. nenhum omega é maior que a capacidade
for i in range(len(points)):
    model.addConstr(omega[i] <= max_capacity, name=f"c8_{i}")
#9. omega da base final é o peso do drone
model.addConstr(omega[base_end] == drone_weight, name="c9")
# 10. Calculo de omega
for i in range(1, n+1):
    model.addConstr(omega[i] == p[i] + quicksum((omega[j])*x[i,j] for j in range(len(points))), name=f"c10_{i}")
# 11. Calculo de e inicial
for j in range(len(points)):
    model.addConstr(e[base_start, j] == battery_capacity - (((d[base_start, j])) * ((omega[j]) * x[base_start, j])), name=f"c11_{j}")
#12. Calculo de e
for i in range(1, n+2):
    for j in range(len(points)):
        model.addConstr(e[i, j] == quicksum(e[k,i]*x[k,i] for k in range(len(points))) - (((d[i, j])) * ((omega[j]) * x[i, j])), name=f"c12_{i}_{j}")
#13. e deve ser maior ou igual a 0
for i in range(len(points)):
    for j in range(len(points)):
        model.addConstr(e[i, j] >= 0, name=f"c13_{i}_{j}")

model.setObjective(quicksum(x[i,j] * (d[i,j]) * omega[j] for i in range(len(points)) for j in range(len(points))), GRB.MINIMIZE)


model.optimize()









In [ ]:
for var in model.getVars():
    if(var.x > 0):
        print(f"{var.varName} = {var.x}")


#print solution objective value
print(f"Objetivo: {model.objVal}")

In [ ]:
subroutes = []
goesTo = {}
for var in model.getVars():
    if var.VarName[0] == 'x' and var.x > 0:
        #print(f"{var.VarName} = {var.x}")
        x,y = var.VarName.split('[')[1].split(']')[0].split(',')
        x = int(x)
        y = int(y)
        if x == base_start:
            subroutes.append([y])
        else:
            goesTo[x] = y

for i in range(len(subroutes)):
    next = goesTo[subroutes[i][-1]]
    while next != base_end:
        subroutes[i].append(next)
        next = goesTo[next]

print(subroutes)
route = []
for subroute in subroutes:
    route += subroute
    route.append(0)
route.pop()
print(route)
print(model.ObjVal)
        